In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.ensemble import GradientBoostingRegressor
from collections import defaultdict


/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/colinshanahan/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
from importlib import reload
from data_cleaning import data_cleaning as dc
from models import contract_regressor as cr

In [17]:
reload(dc)

<module 'data_cleaning.data_cleaning' from '/Users/colinshanahan/galvanize/nhl-contract-predictions/data_cleaning/data_cleaning.py'>

In [49]:
def clean_FA():
    df = pd.read_csv('../data/free_agents_2019.csv')
    df['signing_status'] = df['ufa_status'].apply(lambda x: 1 if 'UFA' else 0)
    df = df[(df.position != 'G') & (df.position != 27) & (df.position != 30)]
    df['forward'] = (df.position != 'D').replace([True, False], [1, 0])
    df['Season_Player'] = '2018 ' + df['player']
    df.set_index(df['Season_Player'], inplace=True)
    df.drop('Season_Player', axis=1, inplace=True)
    fa = df[['player', 'age', 'forward', 'signing_status']]
    return fa

In [51]:
df = clean_FA()
df.sample(5)

,player,age,forward,signing_status
Season_Player,,,,
2018 Curtis Lazar,Curtis Lazar,24,1,1
2018 Jordan Weal,Jordan Weal,26,1,1
2018 Travis Konecny,Travis Konecny,22,1,1
2018 Brad Malone,Brad Malone,29,1,1
2018 Alex Chiasson,Alex Chiasson,28,1,1
